## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

In [25]:
pip install pgeocode


Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install geopy


Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

In [29]:
import urllib.request
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

from bs4 import BeautifulSoup
soup = BeautifulSoup(page,"lxml")
right_table=soup.find("table", attrs={"cellpadding":"2","cellspacing":"0","rules":"all"})

A=[]
B=[]
C=[]
D=[]


for row in right_table.findAll('tr'):
    for cell in row.findAll('td'):
        D.append(cell.text.replace('\n', '').strip())

borough=['Downtown Toronto','North York','East York','Scarborough','East Toronto','Etobicoke','West Toronto'
         ,'Central Toronto','York',"Queen's Park",'Mississauga']
for i in D:
    for j in borough:
        if j in i:
            A.append(i[0:3])
            B.append(j)
            C.append(i[3:].replace('\n','').replace(j,'').replace('(','').replace(')','').replace('/',','))
            break
            

C[4]=C[4][3:] 

#### Tranform the data into a *pandas* dataframe

In [30]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood'] 

# instantiate the dataframe
df = pd.DataFrame(columns=column_names)
df['PostalCode']=A
df['Borough']=B
df['Neighborhood']=C

latitude=[]
longitude=[]
import pgeocode
nomi = pgeocode.Nominatim('CA')
for i in df['PostalCode']:
    latitude.append(nomi.query_postal_code(i).latitude)
    longitude.append(nomi.query_postal_code(i).longitude)

df["Latitude"]=latitude
df["Longitude"]=longitude
df.loc[76,'Latitude']=43.6370
df.loc[76,'Longitude']=-79.6158

#### Create a map of Toronto with neighborhoods superimposed on top.

In [31]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latd = location.latitude
longd = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latd, longd))


# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latd, longd], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [32]:
toronto_data = df
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889


#### Define Foursquare Credentials and Version

In [43]:
CLIENT_ID = 'WMLZND1ZEOVKRPVER4N52WPROB1OM2ZYM4L24NATIXFX34X1' # your Foursquare ID
CLIENT_SECRET = '0WU1VWRC0BFHMHEFW2X2HU4QYVVSNKQDA5PEVG0HQQULDUIM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WMLZND1ZEOVKRPVER4N52WPROB1OM2ZYM4L24NATIXFX34X1
CLIENT_SECRET:0WU1VWRC0BFHMHEFW2X2HU4QYVVSNKQDA5PEVG0HQQULDUIM


## 2. Explore Neighborhoods in Toronto

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [45]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
names=df['Neighborhood']
latitudes=df['Latitude']
longitudes=df['Longitude']
toronto_venues = getNearbyVenues(names,latitudes,longitudes)



Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don MillsNorth
Parkview Hill , Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don MillsSouthFlemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
 Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
East TorontoThe Danforth  East
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview , East Birchmount Park
B

#### Let's find out how many unique categories can be curated from all the returned venues

In [46]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 264 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [50]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]



#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [51]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

#### Let's print each neighborhood along with the top 5 most common venues and put into dataframe

In [52]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

---- Village----
                 venue  freq
0                  Spa   0.5
1        Grocery Store   0.5
2   Miscellaneous Shop   0.0
3        Movie Theater   0.0
4  Monument / Landmark   0.0


----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1               Skating Rink   0.2
2            Badminton Court   0.2
3        Shanghai Restaurant   0.2
4  Latin American Restaurant   0.2


----Alderwood , Long Branch----
         venue  freq
0  Pizza Place  0.14
1          Gym  0.14
2     Pharmacy  0.14
3          Pub  0.14
4  Coffee Shop  0.14


----Bathurst Manor , Wilson Heights , Downsview North----
                       venue  freq
0                Pizza Place  0.17
1                Coffee Shop  0.17
2   Mediterranean Restaurant  0.17
3        Fried Chicken Joint  0.17
4  Middle Eastern Restaurant  0.17


----Bayview Village----
                venue  freq
0         Flower Shop  0.25
1                Park  0.25
2         Gas Station  0.25
3          

                  venue  freq
0           Pizza Place  0.09
1           Coffee Shop  0.06
2  Fast Food Restaurant  0.06
3                  Bank  0.06
4        Medical Center  0.03


----Harbourfront East , Union Station , Toronto Islands----
             venue  freq
0      Music Venue  0.25
1             Park  0.25
2             Café  0.25
3  Harbor / Marina  0.25
4    Metro Station  0.00


----High Park , The Junction South----
                                      venue  freq
0  Residential Building (Apartment / Condo)  0.33
1                             Bowling Alley  0.33
2                                      Park  0.33
3                               Yoga Studio  0.00
4                        Miscellaneous Shop  0.00


----Hillcrest Village----
                                      venue  freq
0  Residential Building (Apartment / Condo)   0.5
1                                      Park   0.5
2                               Yoga Studio   0.0
3                        Miscellaneous 

                venue  freq
0         Coffee Shop  0.08
1                Café  0.05
2          Restaurant  0.05
3  Seafood Restaurant  0.05
4  Italian Restaurant  0.04


----St. James Town , Cabbagetown----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.07
2         Pizza Place  0.05
3  Italian Restaurant  0.05
4          Restaurant  0.05


----Steeles West , L'Amoreaux West----
                  venue  freq
0    Chinese Restaurant  0.15
1  Fast Food Restaurant  0.15
2  Other Great Outdoors  0.08
3          Burger Joint  0.08
4                  Bank  0.08


----Stn A PO Boxes25 The EsplanadeEnclave of M5E----
                venue  freq
0         Coffee Shop  0.16
1          Restaurant  0.07
2                Café  0.04
3               Hotel  0.04
4  Italian Restaurant  0.03


----Studio District----
             venue  freq
0  Coworking Space  0.12
1             Park  0.12
2    Garden Center  0.12
3              Gym  0.12
4      Coffee Shop  0.12


---

First, let's write a function to sort the venues in descending order.

In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Village,Spa,Grocery Store,Women's Store,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant
1,Agincourt,Skating Rink,Breakfast Spot,Badminton Court,Shanghai Restaurant,Latin American Restaurant,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
2,"Alderwood , Long Branch",Sandwich Place,Pharmacy,Convenience Store,Pizza Place,Gym,Pub,Coffee Shop,Women's Store,Doner Restaurant,Donut Shop
3,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Pizza Place,Mediterranean Restaurant,Fried Chicken Joint,Middle Eastern Restaurant,Deli / Bodega,Women's Store,Eastern European Restaurant,Electronics Store,Empanada Restaurant
4,Bayview Village,Gas Station,Flower Shop,Park,Trail,Empanada Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [55]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 3, 1, 1, 1, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [56]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,3.0,Spa,Park,Food & Drink Shop,Women's Store,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
1,M4A,North York,Victoria Village,43.7276,-79.3148,1.0,Hockey Arena,Portuguese Restaurant,Financial or Legal Service,Intersection,Coffee Shop,Pizza Place,Park,Farmers Market,Falafel Restaurant,Event Space
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626,1.0,Coffee Shop,Restaurant,Yoga Studio,Theater,Italian Restaurant,Food Truck,Event Space,Electronics Store,Mexican Restaurant,Distribution Center
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.7223,-79.4504,1.0,Clothing Store,Coffee Shop,Electronics Store,Women's Store,Sandwich Place,Restaurant,Pharmacy,Sushi Restaurant,Cosmetics Shop,Bakery
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889,1.0,Italian Restaurant,Gym,Coffee Shop,Chinese Restaurant,Beer Bar,Boutique,Creperie,Seafood Restaurant,Burger Joint,Burrito Place


Finally, let's visualize the resulting clusters

In [59]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latd = location.latitude
longd = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latd, longd))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [67]:
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].fillna(0)
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype(int)

In [69]:
# create map
map_clusters = folium.Map(location=[latd, longd], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

#### Cluster 1

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,Central Toronto,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,Scarborough,0,Convenience Store,Auto Garage,Women's Store,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant
95,Scarborough,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 2

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Hockey Arena,Portuguese Restaurant,Financial or Legal Service,Intersection,Coffee Shop,Pizza Place,Park,Farmers Market,Falafel Restaurant,Event Space
2,Downtown Toronto,1,Coffee Shop,Restaurant,Yoga Studio,Theater,Italian Restaurant,Food Truck,Event Space,Electronics Store,Mexican Restaurant,Distribution Center
3,North York,1,Clothing Store,Coffee Shop,Electronics Store,Women's Store,Sandwich Place,Restaurant,Pharmacy,Sushi Restaurant,Cosmetics Shop,Bakery
4,Queen's Park,1,Italian Restaurant,Gym,Coffee Shop,Chinese Restaurant,Beer Bar,Boutique,Creperie,Seafood Restaurant,Burger Joint,Burrito Place
5,Etobicoke,1,Pharmacy,Grocery Store,Bank,Skating Rink,Park,Convenience Store,Dog Run,Financial or Legal Service,Field,Fast Food Restaurant
8,East York,1,Pizza Place,Bank,Bus Line,Breakfast Spot,Café,Gastropub,Intersection,Fast Food Restaurant,Pet Store,Pharmacy
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Café,Japanese Restaurant,Italian Restaurant,Lingerie Store,Pizza Place,Theater
10,North York,1,Italian Restaurant,Grocery Store,Japanese Restaurant,Latin American Restaurant,Ice Cream Shop,Asian Restaurant,Gas Station,Pizza Place,Mediterranean Restaurant,Fast Food Restaurant
11,Etobicoke,1,Pizza Place,Construction & Landscaping,Chinese Restaurant,Coffee Shop,Tea Room,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Distribution Center
14,East York,1,Asian Restaurant,Convenience Store,Spa,Video Store,Diner,Beer Store,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant


#### Cluster 3

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,2,Baseball Field,Women's Store,Doner Restaurant,Fish & Chips Shop,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


#### Cluster 4

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Spa,Park,Food & Drink Shop,Women's Store,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
7,North York,3,Pool,Park,Gym,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
13,North York,3,Gym,River,Home Service,Park,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
21,York,3,Park,Women's Store,Mexican Restaurant,Market,Bakery,Gym,Sporting Goods Shop,Beer Store,Eastern European Restaurant,Electronics Store
27,North York,3,Residential Building (Apartment / Condo),Park,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space
35,East York,3,Park,Convenience Store,Intersection,Greek Restaurant,Women's Store,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
36,Downtown Toronto,3,Park,Café,Harbor / Marina,Music Venue,Women's Store,Empanada Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
39,North York,3,Gas Station,Flower Shop,Park,Trail,Empanada Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
46,North York,3,Pool,Park,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Women's Store
61,Central Toronto,3,Photography Studio,Park,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space


#### Cluster 5

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,4,Bar,Women's Store,Doner Restaurant,Fish & Chips Shop,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
